In [15]:
import os
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline
import time
from torch.autograd import Variable

import cv2
import torch
import numpy as np
from facenet_pytorch import MTCNN

import matplotlib.pyplot as plt

from PIL import Image

import glob
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

import torch.optim as optim

import pyautogui

In [16]:
def print_face(img,detections):
    img_with_dets = img.copy()

    x, y, x1, y1 = detections[0][0]
    keypoints =detections[1][0]
    cv2.rectangle(img_with_dets, (x,y), (x1,y1), (0,155,255), 2)
    
    cv2.imshow('Face', img_with_dets)
    cv2.waitKey(0)

In [17]:
def digit_to_classname(digit):
        if digit == 0:
            print('привет!')
            return 'palm' # ладонь
        elif digit == 1:
            image = pyautogui.screenshot(region=(0,0, 300, 400))
            image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
            cv2.imshow('Screen', image)
            cv2.waitKey(0)
            return 'l'
        elif digit == 2:
            
            return 'first'
        elif digit == 3:
            
            return 'fist_moved'
        elif digit == 4:
            
            return 'thumb'
        elif digit == 5:
            
            return 'index'
        elif digit == 6:
            
            return 'ok'
        elif digit == 7:
            print('У меня тоже все ок!')
            return 'palm_moved'
        elif digit == 8:
            
            return 'c'
        elif digit == 9:
            return 'down'

In [18]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 32, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(79680, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [19]:
model = Net()
model.load_state_dict(torch.load('./models/hand_detection_model_state_30_epoch.pth'))

<All keys matched successfully>

In [22]:
vc = cv2.VideoCapture(0)

plt.ion()
detector = MTCNN()

a = 0

is_capturing = vc.isOpened()

has_face = False

if not is_capturing:
    print("Error opening video")


while is_capturing:
    try:
        if not has_face:
            is_capturing, frame = vc.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    
            
            detections = detector.detect(frame)
            print_face(frame, detections)
            has_face = True
        else:
            print("теперь покажите жест")
            plt.pause(5)
            is_capturing, frame = vc.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            frame = frame/255
            
            img = cv2.resize(frame,(170,64))
            torch_img = torch.from_numpy(img).unsqueeze(0).float()
 
            cv2.imshow('Gesture', frame)
            cv2.waitKey(0)            
    
            res = model(torch_img[None, ...])
            res = digit_to_classname(res.argmax())
            print('Жест: ', res)

        a += 1
        if a == 3:
            break

        try:    
            plt.pause(1)
        except Exception:
            pass
    except KeyboardInterrupt:
        vc.release()
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

теперь покажите жест
привет!
Жест:  palm
теперь покажите жест
Жест:  l
